# Tutorial 2 - Data Preprocessing

##### The following notebook is a modified version version of the talk given by April Chen in 2016 titled Depy 2016 Talk: Pre-Modeling: Data Preprocessing and Feature Exploration in Python. More information can be found from the original repo: https://github.com/aprilypchen/depy2016


### The objective of the tutorial is to show the importance of data manipulation, and how to apply common data pre-processing techniques to improve model performance.

In [7]:
# Bread and butter libraries to deal with dataframes and matrices, pandas and numpy

### FILL IN ###

##### For the following workshop, we will be using an edited version of the "adult" dataset from the public UCI repository. The dataset consists of information on various individuals, including age, education, marital status, gender, and income.

In [4]:
# Machine learning models cannot deal with null values, we will go over techniques to deal with them
na_values = ['#NAME?']
df = pd.read_csv('adult.csv', na_values=na_values)

In [6]:
# peek at first 10

### FILL IN ###

#### Binary Classification Problem: Predict, based on various features from the dataset, if someone's income is greater or less than 50k

In [ ]:
# Observe dataset distribution, in practice when unbalanced datasets are not well treated, performance metrics can
# be very misleading
df['income'].value_counts()

In [ ]:
# Encode as 0 if income <=50K and as 1 if income >50K

### FILL IN ###

In [ ]:
# Inspect encoded values for income

### FILL IN ###

## 1. Benchmark performance with unprocessed data

In [ ]:
# Drop na values to not throw any errors
df_unprocessed = df # create a copy of df, save original for later
df_unprocessed = df_unprocessed.dropna(axis=0, how='any')

print(df.shape)
print(df_unprocessed.shape)

In [ ]:
# Remove non-numeric columns so model doesn't throw errors
# Potential loss of information from categorical features is evident

for col_name in df_unprocessed.columns:
    if df_unprocessed[col_name].dtypes not in ['int32','int64','float32','float64']:
    ### FILL IN ###

In [8]:
# Split into features and target variable

### FILL IN ###

In [9]:
# Inspect first 5 values of X

### FILL IN ###

### Import algo to measure baseline accuracy

In [ ]:
# Import common ML tools from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Split data into train/test split

In [ ]:
X_train_unproc, X_test_unproc, y_train, y_test = train_test_split(
    X_unprocessed, y_unprocessed, train_size=0.70, test_size=0.30, random_seed=1)

In [ ]:
# Function that returns model accuracy
def find_model_acc(X_train, y_train, X_test, y_test):
    ### FILL IN ###
    return acc

In [ ]:
acc_unprocessed= find_model_acc(X_train_unproc, y_train, X_test_unproc, y_test)

In [ ]:
acc_unprocessed

#### NOTE: when feeding a test features into your prediction model, ensure test set has gone through the same preprocessing as your training set

## 2. Explore feature space to determine how to perform data pre-processing, then feed processed data into model to evaluate performance difference

In [ ]:
# Separate features from target var. , feature matrix commonly notated as X, and target variable as y

y = df.income
X = df.drop(['income'], 1)

In [ ]:
# optional sanity check
y.head(10)

In [ ]:
X.head(10)

### Dealing with Categorical Data: One-hot encoding

#### One simple strategy to convert categorical data to numerical data is to create one dummy variable for each categorical value possible, then flagging with a 1 when the value is present

In [ ]:
### FILL IN ###

#### Determine how many possibles values there are for each feature

In [ ]:
# Decide which categorical variables you want to use in model 
for col_name in X.columns:
    if X[col_name].dtypes == 'object':
        unique_cat = len(X[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} unique categories".format(col_name=col_name, unique_cat=unique_cat))

In [ ]:
# Investigate why there is a high number of unique values for 'native_country'
X['native_country'].value_counts().sort_values(ascending=False).head(10)

In [ ]:
# In this case, bucket low frequecy categories as "Other"
X['native_country'] = ['United-States ' if x == 'United-States' else 'Other' for x in X['native_country']]

print(X['native_country'].value_counts().sort_values(ascending=False))

#### Create list of important categorical features to encode

In [ ]:
# Create a list of features to dummy# Create 
todummy_list = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']

In [ ]:
# Function to one-hot encode all the categorical variables used for modeling

def dummy_df(df, todummy_list):
    for x in todummy_list:
        dummies = pd.get_dummies(df[x], prefix=x, dummy_na=False)
        df = df.drop(x, 1)
        df = pd.concat([df, dummies], axis=1)
    return df

In [ ]:
X = dummy_df(X, todummy_list)

In [ ]:
# Inspect X

### FILL IN ###

### Investigate null values

In [ ]:
X.isnull().sum().sort_values(ascending=False).head()

In [ ]:
# Impute missing values using Imputer in sklearn.preprocessing
from sklearn.preprocessing import Imputer

In [ ]:
imp = Imputer(missing_values='NaN', strategy='median', axis=0) # you can try with different strategies, based on dataset distribution

In [ ]:
### FILL IN ###
X = pd.DataFrame(data=imp.transform(X) , columns=X.columns)

In [ ]:
# Sanity check
X.isnull().sum().sort_values(ascending=False).head()

### PCA to find most important components

In [ ]:
# Use PCA from sklearn.decompostion to find principal components
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
X_pca = pd.DataFrame(pca.fit_transform(X))

In [ ]:
X_pca.head(10)

### Evaluate the same algorithm, but with preprocessed dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, train_size=0.70, test_size=0.30,random_state=1)

In [ ]:
# find model accuracy on processed data
### FILL IN ###

In [ ]:
acc_proc

In [ ]:
improvement = np.round(((acc_proc - acc_unprocessed)/acc_unprocessed),3)*100

In [ ]:
improvement